This is a short notebook to explore alternative ways of calculating curve length. Given a curve $\gamma : [0,1] \rightarrow \mathbb R^d$ we define it's length on an isotropic Riemannian manifold as
$$L[\gamma] = \int_0^1a(\gamma(t))\|\gamma'(t)\| dt.$$
Where $0 < \alpha_1 \leq a \leq \alpha_2$.
Suppose that our approximation of $\gamma$, denoted $\tilde \gamma$, is the piecewise linear interpolation of the points $\{\mathbf x_i\}_{i=0,...,N} \subset \mathbb R^d$. To define an explicit parameterisation fix $i \in \{1,...,N\}$. Then we have
$$\tilde \gamma(t) = N(\mathbf x_i - \mathbf x_{i-1})\left(t - \frac{i-1}{N}\right) + \mathbf x_{i-1} \text{ in } t \in \left[ \frac{i-1}{N}, \frac{i}{N} \right).$$
Consequently, we have, by splitting the integral
$$L[\tilde \gamma] = \sum_{i=1}^N \int_{(i-1)/N}^{i/N}a(\tilde \gamma(t))\|\tilde \gamma'(t)\| dt. $$
Using the explicit parameterisation for $\tilde \gamma$
$$\int_{(i-1)/N}^{i/N}a(\tilde \gamma(t))\|\tilde \gamma'(t)\| dt = \int_{(i-1)/N}^{i/N}a(\tilde \gamma(t))\|N(\mathbf x_i - \mathbf x_{i-1})\| dt \\= N\|\mathbf x_i - \mathbf x_{i-1}\|\int_{(i-1)/N}^{i/N}a(\tilde \gamma(t)) dt.$$
Denote the composite uniform trapezoidal rule as $T(M,x_0,x_1,f)$ where $1/M$ is the mesh size, $x_0, x_1$ are the limits of the integral and $f$ is the integrand. As the leading term of the error for the composite trapezoial rule, on the $i^{th}$ interval is
$$-\frac{1}{2M^2}\left(\frac{i}{N} - \frac{i-1}{N}\right)^2\left((a \circ \tilde \gamma)'(i/N)-(a \circ \tilde \gamma)'((i-1)/N)\right) \\ =-\frac{1}{2M^2N^2}\left((a \circ \tilde \gamma)'(i/N)-(a \circ \tilde \gamma)'((i-1)/N)\right).$$
Since
$$(a \circ \tilde \gamma)'(x)=a'(\tilde \gamma(x)) \cdot \tilde \gamma'(x)$$
it follows that
$$(a \circ \tilde \gamma)'(i/N)-(a \circ \tilde \gamma)'((i-1)/N) \\= a'(\tilde \gamma(i/N)) \cdot \tilde \gamma'(i/N) - a'(\tilde \gamma((i-1)/N)) \cdot \tilde \gamma'((i-1)/N)\\= a'(\mathbf x_i) \cdot \tilde \gamma'(i/N) - a'(\mathbf x_{i-1}) \cdot \tilde \gamma'((i-1)/N) \\=a'(\mathbf x_i) \cdot N(\mathbf x_{i+1} - \mathbf x_{i}) - a'(\mathbf x_{i-1}) \cdot N(\mathbf x_i - \mathbf x_{i-1}) \\= \left(a'(\mathbf x_i) - a'(\mathbf x_{i-1})\right) \cdot N(\mathbf x_i - \mathbf x_{i-1}).$$
There error therefore becomes
$$-\frac{1}{2M^2N^2}\left(a'(\mathbf x_i) - a'(\mathbf x_{i-1})\right) \cdot N(\mathbf x_i - \mathbf x_{i-1}).$$
The length is therefore
$$L[\tilde \gamma] = \sum_{i=1}^N N\|\mathbf x_i - \mathbf x_{i-1}\|T\left(M,\frac{i-1}{N},\frac{i}{N},a \circ \tilde \gamma \right) + O(N/M^2)$$

$$\bar{y}(t) = \mathbf{x_{i-1}} + t(\mathbf{x}_i - \mathbf{x}_{i-1}) \, \mathrm{in}\, t \in [0, 1)$$

In [1]:
from typing import Callable
import numpy as np


def approximate_integral_line_segment(line_segment_index: int,
                                      line_segment_count: int,
                                      trapezoid_count: int,
                                      a: Callable[[np.ndarray], np.ndarray]) -> float:
    """
    Args:
        line_segment_index: i
        trapezoid_count: M
        line_segment_count: N
        a: \mathbb{R} \rightarrow \mathbb{R}
    """
    start_time = (line_segment_index - 1) / line_segment_count
    stop_time = line_segment_index / line_segment_count
    ts = np.linspace(start_time, stop_time, trapezoid_count)  # shape: (trapezoid_count,)
    ys = a(ts)
    return np.trapz(ys, ts)

In [2]:
N = 20  # number of points on the global curve
line_segment_count = 9  # M
dimensionality = 5
xs = np.random.randn(N, dimensionality)
diff = xs[1:] - xs[:-1]
interpolation_factors = np.linspace(0, 1, line_segment_count)
# diff:                                 shape(N-1, dimensionality)
# diff[..., np.newaxis, :]:             shape(N-1, 1, dimensionality)
# interpolation_factors[:, np.newaxis]: shape(line_segment_count, 1)
# xs[:-1, ..., np.newaxis, :]:          shape(N-1, 1, dimensionality)
line_segment_things = diff[..., np.newaxis, :] * interpolation_factors[:, np.newaxis] + xs[:-1, ..., np.newaxis, :]
line_segment_things.shape

def metric_fn(xs: np.ndarray, beta=0.6, weights=None):
    if weights is None:
        weights = np.ones(xs.shape[-1])
    return np.exp(-beta * (xs @ weights))

print(line_segment_things.shape)
metric_values = metric_fn(line_segment_things)

# length of curve
np.trapz(metric_values, dx=(1 / line_segment_count), axis=-1).sum()

(19, 9, 5)


41.65026329112853

In [3]:
approximate_integral_line_segment(1, 1, 200, lambda x: x**2)

0.3333375419812631